# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from math import sin, cos, sqrt, atan2, radians

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load (Remember to Change These)
file_to_load = '..\Output\Weather_Data.csv'

# Read Purchasing File and store into Pandas data frame
weather_data = pd.read_csv(file_to_load)
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ushuaia,40,AR,1585710920,75,-54.80,-68.30,8.00,3.10
1,Upernavik,85,GL,1585711150,87,72.79,-56.15,-20.18,6.13
2,Cape Town,20,ZA,1585710900,67,-33.93,18.42,17.22,8.70
3,Arraial do Cabo,12,BR,1585710888,83,-22.97,-42.03,24.77,3.64
4,Atuona,1,PF,1585711151,80,-9.80,-139.03,27.78,6.08
...,...,...,...,...,...,...,...,...,...
577,Saint-François,0,FR,1585711307,73,46.42,3.91,-3.00,1.50
578,Palmas,75,BR,1585711308,88,-10.21,-48.36,26.00,1.50
579,Charters Towers,0,AU,1585711308,33,-20.10,146.27,30.03,6.14
580,Cockburn Town,0,TC,1585711308,72,21.46,-71.14,24.88,4.25


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
figure_layout={
    'width': '800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'marging': '0 auto 0 auto'
}
gmaps.configure(api_key=g_key)
fig=gmaps.figure(layout=figure_layout)
fig


Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='800px'))

In [4]:
coordinates= weather_data[['Lat','Lng']]
humidity=weather_data['Humidity'].astype(float)
figure_layout={
    'width': '800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'marging': '0 auto 0 auto'
}
gmaps.configure(api_key=g_key)
fig=gmaps.figure(layout=figure_layout)
heat_layer=gmaps.heatmap_layer(coordinates,weights=humidity,dissipating=False,max_intensity=weather_data['Humidity'].max(),point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
filtered_data = weather_data[(weather_data['Max Temp'] >= 20) & (weather_data['Max Temp'] <= 30) & (weather_data['Humidity'] >= 80) & (weather_data['Cloudiness'] <= 10)]
narrowed_data=filtered_data.dropna().set_index('City').reset_index()
narrowed_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Atuona,1,PF,1585711151,80,-9.80,-139.03,27.78,6.08
1,Guarujá,8,BR,1585711158,82,-23.99,-46.26,24.44,1.50
2,San Ignacio,10,BZ,1585711184,88,17.16,-89.07,24.44,1.34
3,Barra de Tecoanapa,0,MX,1585711188,80,16.52,-98.75,25.95,1.83
4,Sharjah city,0,AE,1585711105,82,25.36,55.40,23.00,2.10
5,Abu Dhabi,0,AE,1585711023,88,24.47,54.37,23.89,2.10
6,Manzanillo,0,CU,1585711239,81,20.34,-77.12,23.45,2.25
7,Sambava,5,MG,1585711265,89,-14.27,50.17,24.85,1.81
8,Agua Dulce,1,MX,1585711308,84,18.13,-94.13,25.00,4.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

names=[]
cities=[]
countries=[]
latitudes=[]
longitudes=[]


for x in range(len(narrowed_data['Lat'])):
    coordinates=f"{str(narrowed_data['Lat'][x])},{str(narrowed_data['Lng'][x])}"
    params = {
        "location": coordinates,
        "radius": 5000,
        "type": "lodging",
        "keyword": "hotel" or "motel" or "inn" or "hostal",
        "key": g_key,}
    
    
    try:
        response = requests.get(base_url, params=params).json()
        names.append(response['results'][0]['name'])
        cities.append(narrowed_data['City'][x])
        countries.append(narrowed_data['Country'][x])
        latitudes.append(response['results'][0]['geometry']['location']['lat'])
        longitudes.append(response['results'][0]['geometry']['location']['lng'])
    except:
        names.append(np.nan)
        cities.append(narrowed_data['City'][x])
        countries.append(narrowed_data['Country'][x])
        latitudes.append(np.nan)
        longitudes.append(np.nan)

hotel_data = {
    
    "City": cities,
    "Country": countries,
    "Hotel Name": names,
    "Lat": latitudes,
    "Lng": longitudes,
}

hotel_df = pd.DataFrame(hotel_data)
hotel_df

,City,Country,Hotel Name,Lat,Lng
0,Atuona,PF,Simplicité Marquises,-9.807877,-139.030335
1,Guarujá,BR,Casa Grande Hotel Resort & Spa,-23.988681,-46.240903
2,San Ignacio,BZ,San Ignacio Resort Hotel,17.153193,-89.068523
3,Barra de Tecoanapa,MX,Hotel Garapacho,16.555608,-98.788328
4,Sharjah city,AE,Citymax Hotel Sharjah,25.336769,55.400419
5,Abu Dhabi,AE,Grand Hyatt Abu Dhabi Hotel & Residences Emira...,24.458391,54.320636
6,Manzanillo,CU,Hotel Guacanayabo,20.336912,-77.134666
7,Sambava,MG,Carrefour,-14.257901,50.160545
8,Agua Dulce,MX,Hotel Antares,18.116634,-94.142347


In [7]:
hotel_df=hotel_df.dropna()
hotel_df

,City,Country,Hotel Name,Lat,Lng
0,Atuona,PF,Simplicité Marquises,-9.807877,-139.030335
1,Guarujá,BR,Casa Grande Hotel Resort & Spa,-23.988681,-46.240903
2,San Ignacio,BZ,San Ignacio Resort Hotel,17.153193,-89.068523
3,Barra de Tecoanapa,MX,Hotel Garapacho,16.555608,-98.788328
4,Sharjah city,AE,Citymax Hotel Sharjah,25.336769,55.400419
5,Abu Dhabi,AE,Grand Hyatt Abu Dhabi Hotel & Residences Emira...,24.458391,54.320636
6,Manzanillo,CU,Hotel Guacanayabo,20.336912,-77.134666
7,Sambava,MG,Carrefour,-14.257901,50.160545
8,Agua Dulce,MX,Hotel Antares,18.116634,-94.142347


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Display Map
coordinates= weather_data[['Lat','Lng']]
humidity=weather_data['Humidity'].astype(float)
figure_layout={
    'width': '800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'marging': '0 auto 0 auto'
}
gmaps.configure(api_key=g_key)
fig=gmaps.figure(layout=figure_layout)
heat_layer=gmaps.heatmap_layer(coordinates,weights=humidity,dissipating=False,max_intensity=weather_data['Humidity'].max(),point_radius=4)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='800px'))